In [2]:
import pickle
with open('./datasets/dataset_sz_grid/GPS/taxi/0_6_gps.pkl', 'rb') as f:
    data = pickle.load(f, encoding="bytes")

In [3]:
data

,id,lon,lat,time,speed,direction
12980,粤B0U8X0,909,218,2018-10-01T05:28:06.000Z,0,164
59555,粤B408YP,908,138,2018-10-01T09:55:47.000Z,24,149
99992,粤B6K4S9,805,102,2018-10-01T01:29:29.000Z,16,75
101068,粤B6L5E0,777,69,2018-10-01T14:41:13.000Z,15,294
111797,粤B7C6R5,846,193,2018-10-01T03:03:59.000Z,61,73
...,...,...,...,...,...,...
131720190,粤BDP7850,819,138,2019-10-01T05:29:19.000Z,0,151
131724485,粤BDQ0515,846,172,2019-10-01T02:09:46.000Z,0,71
131724486,粤BDQ0515,846,172,2019-10-01T02:12:13.000Z,0,71
131734491,粤BDQ8371,842,182,2019-10-01T03:00:25.000Z,0,71


In [7]:
with open('./dataset/GPS/beijing_gps_dir_speed_interval_sorted.pkl', 'rb') as f:
    data2 = pickle.load(f, encoding="bytes")

In [8]:
data2

,ID,time,lat,lon,dir,speed,timeinterval
0,0,1228061046,39.71743,116.61815,0,0,NaN
1,0,1228088457,39.71742,116.61798,0,0,177.5
2,0,1228088520,39.71670,116.61420,159,0,150.5
3,0,1228088758,39.71742,116.61798,0,0,272.5
4,0,1228090926,39.71670,116.61428,0,0,354.5
...,...,...,...,...,...,...,...
50041852,28036,1228742522,39.95209,116.30327,23470,0,427.5
50041853,28036,1228742828,39.95206,116.30322,22420,0,429.0
50041854,28036,1228743380,39.95198,116.30315,1820,0,NaN
50041855,28037,1228089568,40.08177,116.23259,16130,1222,NaN


In [8]:
import numpy as np
import pandas as pd
help(np.ndim)
# help(pd.ndim)

Help on function ndim in module numpy:

ndim(a)
    Return the number of dimensions of an array.
    
    Parameters
    ----------
    a : array_like
        Input array.  If it is not already an ndarray, a conversion is
        attempted.
    
    Returns
    -------
    number_of_dimensions : int
        The number of dimensions in `a`.  Scalars are zero-dimensional.
    
    See Also
    --------
    ndarray.ndim : equivalent method
    shape : dimensions of array
    ndarray.shape : dimensions of array
    
    Examples
    --------
    >>> np.ndim([[1,2,3],[4,5,6]])
    2
    >>> np.ndim(np.array([[1,2,3],[4,5,6]]))
    2
    >>> np.ndim(1)
    0



In [2]:
# rename files
import os

def rename_files(directory):
    for filename in os.listdir(directory):
        if filename.endswith("_mask.png"):
            new_name = filename.replace("_mask.png", "_gps.png")
            old_path = os.path.join(directory, filename)
            new_path = os.path.join(directory, new_name)
            os.rename(old_path, new_path)
            print(f'Renamed: {old_path} -> {new_path}')

# Replace 'your_directory' with the path to the directory containing your files
rename_files('datasets/dataset_time/GPS/time_count_patch/')


Renamed: datasets/dataset_time/GPS/time_count_patch/1_56_mask.png -> datasets/dataset_time/GPS/time_count_patch/1_56_gps.png
Renamed: datasets/dataset_time/GPS/time_count_patch/5_21_mask.png -> datasets/dataset_time/GPS/time_count_patch/5_21_gps.png
Renamed: datasets/dataset_time/GPS/time_count_patch/13_31_mask.png -> datasets/dataset_time/GPS/time_count_patch/13_31_gps.png
Renamed: datasets/dataset_time/GPS/time_count_patch/44_54_mask.png -> datasets/dataset_time/GPS/time_count_patch/44_54_gps.png
Renamed: datasets/dataset_time/GPS/time_count_patch/2_19_mask.png -> datasets/dataset_time/GPS/time_count_patch/2_19_gps.png
Renamed: datasets/dataset_time/GPS/time_count_patch/0_10_mask.png -> datasets/dataset_time/GPS/time_count_patch/0_10_gps.png
Renamed: datasets/dataset_time/GPS/time_count_patch/18_57_mask.png -> datasets/dataset_time/GPS/time_count_patch/18_57_gps.png
Renamed: datasets/dataset_time/GPS/time_count_patch/23_50_mask.png -> datasets/dataset_time/GPS/time_count_patch/23_50_

In [17]:
# 2D Wasserstein distance
import numpy as np
from scipy.ndimage import distance_transform_edt
from scipy.optimize import linear_sum_assignment
from tqdm import tqdm

def wasserstein_distance(image1, image2):
    # Compute the Euclidean distance transform of the binary images
    distance_map1 = distance_transform_edt(image1)
    distance_map2 = distance_transform_edt(image2)

    # Compute the cost matrix as the pairwise distances between non-zero pixels
    nonzero_indices1 = np.transpose(np.nonzero(image1))
    nonzero_indices2 = np.transpose(np.nonzero(image2))
    cost_matrix = np.linalg.norm(nonzero_indices1[:, None] - nonzero_indices2, axis=2)

    # Solve the assignment problem to find optimal pixel correspondences
    row_indices, col_indices = linear_sum_assignment(cost_matrix)

    # Calculate the Wasserstein distance
    wasserstein_dist = np.sum(cost_matrix[row_indices, col_indices] *
                              np.maximum(distance_map1[nonzero_indices1[row_indices, 0],
                                                        nonzero_indices1[row_indices, 1]],
                                         distance_map2[nonzero_indices2[col_indices, 0],
                                                        nonzero_indices2[col_indices, 1]]))

    return wasserstein_dist


# # Example usage:
# binary_image1 = np.array([[0, 0, 5],
#                           [1, 0, 0],
#                           [0, 1, 0]])
# binary_image2 = np.array([[0, 1, 0],
#                           [0, 0, 1],
#                           [1, 0, 0]])
# distance = wasserstein_distance(binary_image1, binary_image2)
# print("Wasserstein Distance:", distance)


mask_path = "datasets/sz_emd/mask"
pred_path = "datasets/sz_emd/pred/sat_only"

distance = 0
n = len(os.listdir(pred_path))
for filename in tqdm(os.listdir(pred_path)):
    if filename.endswith(".png"):
        pred_image = cv2.imread(os.path.join(pred_path, filename), cv2.IMREAD_GRAYSCALE)
        mask_image = cv2.imread(os.path.join(mask_path, f"{filename[:-4]}_mask.png"), cv2.IMREAD_GRAYSCALE)
        distance += wasserstein_distance(pred_image, mask_image)
average_distance = distance / n
print(f'Average Wasserstein distance: {average_distance}')

100%|██████████| 528/528 [02:37<00:00,  3.36it/s]

Average Wasserstein distance: 1593.92517488872


In [2]:
# Write average_distance to CSV file
import csv
# File path to save the CSV file
csv_file_path = "./emd.csv"
# Write average_distance to CSV file
average_distance = 0.5
for i in range(10):
    with open(csv_file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([i, average_distance])

In [6]:
# find the filename with the highest epoch number
import os
import re

file_path = "./weights_sz_v1/dlink34_sat_gpsimage_gaussian_ltqs_log__"

# Regular expression pattern to match the epoch number
pattern = r'epoch(\d+)_'

# Initialize variables to store the maximum epoch number and corresponding filename
max_epoch = 0
max_epoch_filename = ""

# Iterate through filenames
for filename in os.listdir(file_path):
    # Check if the filename is not "prediction"
    if filename.endswith(".pth"):
        # Extract epoch number
        match = re.search(pattern, filename)
        if match:
            epoch_number = int(match.group(1))
            # Update max_epoch and max_epoch_filename if the current epoch number is higher
            if epoch_number > max_epoch:
                max_epoch = epoch_number
                max_epoch_filename = filename

# Print the filename with the highest epoch number
print("Filename with the highest epoch number:", max_epoch_filename)

Filename with the highest epoch number: epoch53_val0.6260_test0.9428_0.6776_0.7511_0.5538_0.7096_0.0116.pth


In [ ]:
# Calculate the Wasserstein distance between two 1D arrays

from scipy.stats import wasserstein_distance
u = [0, 1, 2, 3]
v = [0, 1, 2, 3]

dis = wasserstein_distance(u, v, [0,0,1,0], [1, 1, 1, 1])
print(dis)

1.0


In [ ]:
# transpose mask from 1024x1024 to 128x128

import cv2
import os
import sys
sys.path.append('..')

def transpose_image(input_path, output_path):
    # Read the input image
    img = cv2.imread(input_path)

    # Resize the image to 128x128 while maintaining aspect ratio
    resized_img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_AREA)

    # Save the resized image
    cv2.imwrite(output_path, resized_img)

# Example usage:
path = "datasets/dataset_sz_grid/test/mask"
save_path = "datasets/dataset_sz_grid/sz_2d_emd/mask"
iter = tqdm(os.listdir(path))
for filename in iter:
    if filename.endswith(".png"):
        input_image_path = os.path.join(path, filename)
        output_image_path = os.path.join(save_path, filename)
        transpose_image(input_image_path, output_image_path)
        iter.set_description_str(f"Processing {filename}")
print("Done!")

In [59]:
# transpose the prediction of every model to 128x128
import os
from scipy.stats import wasserstein_distance
import cv2
import numpy as np
import csv
from tqdm import tqdm

def transpose_image(input_path, output_path):
    # Read the input image
    img = cv2.imread(input_path)

    # Resize the image to 128x128 while maintaining aspect ratio
    resized_img = cv2.resize(img, (128, 128), interpolation=cv2.INTER_AREA)

    # Save the resized image
    cv2.imwrite(output_path, resized_img)


weights_dirs = ["weights_sz_v1", "weights_sz_v2", "weights_sz_v3"]
models = ["unet", "resunet", "deeplabv3+", "linknet", "dlink34_1d", "dlink34"]
methods = ["_gpsimage_only_gaussian_ltqs_log__", "_sat_gpsimage_gaussian_ltqs_log__"]
# methods = ["_gpsdata_only_count_direct__", "_sat_only____", "_sat_gpsdata_count_direct__"]


for weights_dir in weights_dirs:
    print(f"Transposing pred from 1024 to 128 size for {weights_dir}:")
    for model in models:
        print(f"Model: {model}")
        for method in methods:
            print(f"Method: {method}")
            folder_path = f"./{weights_dir}/{model}{method}"
            pred_path = os.path.join(folder_path, "prediction")
            save_path = os.path.join(folder_path, "prediction_transposed")
            if not os.path.exists(save_path):
                os.makedirs(save_path)
            iter = tqdm(os.listdir(pred_path))
            for filename in iter:
                if filename.endswith(".png"):
                    pred_image_path = os.path.join(pred_path, filename)
                    output_image_path = os.path.join(save_path, filename)
                    transpose_image(pred_image_path, output_image_path)
                    iter.set_description_str(f"Processing {filename}")

print('Done!')


Calculating Wasserstein distance for weights_sz_v1:
Model: unet
Method: _gpsimage_only_gaussian_ltqs_log__


Processing 2011_9.png: 100%|██████████| 172/172 [00:02<00:00, 70.52it/s]


Method: _sat_gpsimage_gaussian_ltqs_log__


Processing 2011_9.png: 100%|██████████| 172/172 [00:02<00:00, 63.33it/s]


Model: resunet
Method: _gpsimage_only_gaussian_ltqs_log__


...

Model: dlink34
Method: _gpsimage_only_gaussian_ltqs_log__


Processing 2011_9.png: 100%|██████████| 172/172 [00:02<00:00, 71.55it/s]


Method: _sat_gpsimage_gaussian_ltqs_log__


Processing 2011_9.png: 100%|██████████| 172/172 [00:02<00:00, 72.54it/s]

Done!


Processing 2003_3_mask.png: 100%|██████████| 172/172 [00:02<00:00, 80.89it/s]

Done!
